# Coletar dados

In [1]:
import tweepy
import pandas as pd
import numpy as np
import requests as rq
import json
import re

### Dados

In [2]:
data_twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')
data_image_archive = pd.read_csv('image-predictions.tsv', "\t")

### Acesso a API

In [3]:
consumer_key = '-'
consumer_secret = '-'
access_token = '-'
access_secret = '-'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)


file = open("tweet_json.txt", "a+") 


for id in data_twitter_archive['tweet_id']:
    ''''
    try:
        twitter = api.get_status(id)
        rt = twitter['retweet_count']
        fv = twitter['favorite_count'] 
        file.write(str(id)+','+str(rt)+','+str(fv)+'\n')
    except Exception as erro:
        pass '''
    
file.close()

In [4]:
twitter_favorites_retweet = pd.read_csv('tweet_json.txt', names = ["tweet_id", "rt", "fv"])

# Avaliar dados

In [5]:
data_twitter_archive.info()
data_twitter_archive.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


- As colunas doggo, floofer, pupper, puppo são referrentes a estágio dos cachorros, esses dados devem ser agrupados em uma coluna.

- Timestamp está com o tipo de dado errado.

In [6]:
data_twitter_archive.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

- Os valores dos numeradores apresentam dados fora do normal

In [7]:
data_twitter_archive.rating_denominator.value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

- Os valores dos denominadores apresentam dados fora do normal

In [8]:
data_twitter_archive.name.value_counts()

None          745
a              55
Charlie        12
Oliver         11
Lucy           11
Cooper         11
Lola           10
Penny          10
Tucker         10
Winston         9
Bo              9
the             8
Sadie           8
Toby            7
Bailey          7
Buddy           7
Daisy           7
an              7
Jax             6
Bella           6
Milo            6
Leo             6
Scout           6
Rusty           6
Oscar           6
Dave            6
Jack            6
Stanley         6
Koda            6
Sammy           5
             ... 
Griswold        1
Odin            1
Marty           1
Lassie          1
Brandonald      1
Chesney         1
Shadoe          1
Moofasa         1
Rolf            1
Berb            1
Vinscent        1
DonDon          1
Iroh            1
Koko            1
Ronduh          1
Mack            1
Ricky           1
Dotsy           1
Lorelei         1
Zeus            1
Saydee          1
Todo            1
Jameson         1
Milky           1
Goose     

- Há nomes incorretos para os cachorros. Ex.: 'a', 'the', 'an' e 'None'. Os que se iniciam com em letra minuscula e o 'None'

In [9]:
data_image_archive.info()
data_image_archive.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


- Há previsões que não são cachorros

In [10]:
twitter_favorites_retweet.info()
twitter_favorites_retweet.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2186 entries, 0 to 2185
Data columns (total 3 columns):
tweet_id    2186 non-null int64
rt          2186 non-null int64
fv          2186 non-null int64
dtypes: int64(3)
memory usage: 51.3 KB


,tweet_id,rt,fv
0,892420643555336193,8836,39455
1,892177421306343426,6475,33767
2,891815181378084864,4296,25427
3,891689557279858688,8915,42831
4,891327558926688256,9708,40978


- Não foi possível buscar as informações para todos os twitters do arquivo disponibilizado, assim o arquivo twitter_favorites_retweet possui menos linhas que o arquivo de data_twitter_archive.

## Qualidade
- Nomes de cachorros que estão incorretos. Ex.: 'a', 'the', 'an', 'None'
- Numerador e denominador possuem valores que fogem do padrão utilizado pela pagina WeRateDogs
- Há twits que são retweets e devem ser retirados, já que queremos somente os twitters originais
- A coluna timestamp, do arquivo data_twitter_archive apresenta o tipo de dado errado.
- Falta imagens para alguns twittes
- Falta os valores de retweets e favorite para alguns twitters
- Nas colunas  doggo, floofer, pupper, puppo 'None' não é tratado como null
- Estão faltando dados de expanded_urls, há somente 2297 valores.

## Arrumação
- No dataframe de twitters as colunas doggo, floofer, pupper, puppo devem ser agrupados em uma só coluna
- A coluna de rating deveria ser única e não dividida em duas (numerador e denominador)
- As tabelas deveriam fazer parte de um único dataset

# Limpar dados

In [11]:
clean_twitters_fv_rt = twitter_favorites_retweet.copy()
clean_twitter_archive = data_twitter_archive.copy()
clean_twitter_image = data_image_archive.copy()

### Definir
- Problema: Há twitters que são retweets e devem ser retirados, já que queremos somente os twitters originais
- Resolução: Retirar registros de retweets e retirar colunas que se referem a eles retweeted_status_id, retweeted_status_user_id e retweeted_status_timestamp

### Código

In [12]:
clean_twitter_archive = clean_twitter_archive[clean_twitter_archive.retweeted_status_id.isnull()]
clean_twitter_archive = clean_twitter_archive.drop(['retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp'],axis=1)

### Testar

In [13]:
clean_twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 14 columns):
tweet_id                 2175 non-null int64
in_reply_to_status_id    78 non-null float64
in_reply_to_user_id      78 non-null float64
timestamp                2175 non-null object
source                   2175 non-null object
text                     2175 non-null object
expanded_urls            2117 non-null object
rating_numerator         2175 non-null int64
rating_denominator       2175 non-null int64
name                     2175 non-null object
doggo                    2175 non-null object
floofer                  2175 non-null object
pupper                   2175 non-null object
puppo                    2175 non-null object
dtypes: float64(2), int64(3), object(9)
memory usage: 254.9+ KB


### Definir
- Problema: As tabelas deveriam fazer parte de um único dataset, Falta imagens para alguns twittes, Falta os valores de retweets e favorite para alguns twitters e falta dados de expanded_urls, há somente 2297 valores.
- Solução: Unir os dataset em uma tabela, garantindo assim um dataset que possui twittes que possuem imagem e a contagem de retweets e favorites. 

### Código

In [14]:
twitter_archive_master = pd.merge(clean_twitter_archive, clean_twitter_image, on='tweet_id', how='inner')
twitter_archive_master = pd.merge(twitter_archive_master, clean_twitters_fv_rt, on='tweet_id', how='inner')

### Testar

In [15]:
twitter_archive_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2136 entries, 0 to 2135
Data columns (total 27 columns):
tweet_id                 2136 non-null int64
in_reply_to_status_id    23 non-null float64
in_reply_to_user_id      23 non-null float64
timestamp                2136 non-null object
source                   2136 non-null object
text                     2136 non-null object
expanded_urls            2136 non-null object
rating_numerator         2136 non-null int64
rating_denominator       2136 non-null int64
name                     2136 non-null object
doggo                    2136 non-null object
floofer                  2136 non-null object
pupper                   2136 non-null object
puppo                    2136 non-null object
jpg_url                  2136 non-null object
img_num                  2136 non-null int64
p1                       2136 non-null object
p1_conf                  2136 non-null float64
p1_dog                   2136 non-null bool
p2                       2

- Em alguns casos há mais de uma imagem para o cada tweet

### Definir
- Problema: No dataframe de twitters as colunas doggo, floofer, pupper, puppo representam o mesmo tipo de dado, nessas colunas o 'None' não é tratado como null 
- Solução: Criar uma coluna para receber esses valores e tratar 'None' como null

### Código

In [16]:
twitter_archive_master.doggo = twitter_archive_master.doggo.replace('None', '')
twitter_archive_master.floofer = twitter_archive_master.floofer.replace('None', '')
twitter_archive_master.pupper =  twitter_archive_master.pupper.replace('None', '')
twitter_archive_master.puppo = twitter_archive_master.puppo.replace('None', '')

twitter_archive_master.floofer.value_counts()
twitter_archive_master.pupper.value_counts()
twitter_archive_master.puppo.value_counts()

twitter_archive_master["estagio"] = twitter_archive_master.floofer.map(str)+twitter_archive_master.pupper.map(str) + twitter_archive_master.puppo.map(str)+twitter_archive_master.doggo.map(str)
twitter_archive_master.estagio = twitter_archive_master.estagio.replace('', np.nan)

twitter_archive_master = twitter_archive_master.drop(['doggo','floofer','pupper', 'puppo'],axis=1)

### Testar

In [17]:
twitter_archive_master.info()
twitter_archive_master.estagio.value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2136 entries, 0 to 2135
Data columns (total 24 columns):
tweet_id                 2136 non-null int64
in_reply_to_status_id    23 non-null float64
in_reply_to_user_id      23 non-null float64
timestamp                2136 non-null object
source                   2136 non-null object
text                     2136 non-null object
expanded_urls            2136 non-null object
rating_numerator         2136 non-null int64
rating_denominator       2136 non-null int64
name                     2136 non-null object
jpg_url                  2136 non-null object
img_num                  2136 non-null int64
p1                       2136 non-null object
p1_conf                  2136 non-null float64
p1_dog                   2136 non-null bool
p2                       2136 non-null object
p2_conf                  2136 non-null float64
p2_dog                   2136 non-null bool
p3                       2136 non-null object
p3_conf                  21

pupper          218
doggo            71
puppo            26
pupperdoggo      11
floofer           7
flooferdoggo      1
puppodoggo        1
Name: estagio, dtype: int64

- Há mais de classificação para os cachorros, para resolver esse problema iremos manter a primeira classificação 

In [18]:
twitter_archive_master.estagio = twitter_archive_master.estagio.replace('pupperdoggo', 'pupper')
twitter_archive_master.estagio = twitter_archive_master.estagio.replace('flooferdoggo', 'floofer')
twitter_archive_master.estagio = twitter_archive_master.estagio.replace('puppodoggo', 'puppo')

In [19]:
twitter_archive_master.estagio.value_counts()

pupper     229
doggo       71
puppo       27
floofer      8
Name: estagio, dtype: int64

### Definir 
- Problema: A coluna timestamp, do arquivo data_twitter_archive apresenta o tipo de dado errado.
- Solução: Realizar um cast na coluna

### Código 

In [20]:
twitter_archive_master.timestamp = pd.to_datetime(twitter_archive_master.timestamp)

### Testar

In [21]:
twitter_archive_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2136 entries, 0 to 2135
Data columns (total 24 columns):
tweet_id                 2136 non-null int64
in_reply_to_status_id    23 non-null float64
in_reply_to_user_id      23 non-null float64
timestamp                2136 non-null datetime64[ns]
source                   2136 non-null object
text                     2136 non-null object
expanded_urls            2136 non-null object
rating_numerator         2136 non-null int64
rating_denominator       2136 non-null int64
name                     2136 non-null object
jpg_url                  2136 non-null object
img_num                  2136 non-null int64
p1                       2136 non-null object
p1_conf                  2136 non-null float64
p1_dog                   2136 non-null bool
p2                       2136 non-null object
p2_conf                  2136 non-null float64
p2_dog                   2136 non-null bool
p3                       2136 non-null object
p3_conf            

### Definir 
- Problema: - Há nomes incorretos para os cachorros. Ex.: 'a', 'the', 'an' e 'None'. Os que se iniciam com em letra minuscula e o 'None'
- Solução: Subustituir dados incorretos por NaN

### Código

In [22]:
twitter_archive_master.name = twitter_archive_master.name.replace('None', np.NaN)
twitter_archive_master.name = twitter_archive_master.name.replace('a', np.NaN)
twitter_archive_master.name = twitter_archive_master.name.replace('the', np.NaN)
twitter_archive_master.name = twitter_archive_master.name.replace('an', np.NaN)

### Testar

In [23]:
twitter_archive_master.name.value_counts()

Charlie     12
Oliver      11
Cooper      11
Tucker      10
Lucy        10
Penny       10
Bo           9
Lola         9
Winston      9
Sadie        8
Buddy        7
Daisy        7
Toby         7
Scout        6
Bella        6
Bailey       6
Jack         6
Jax          6
Koda         6
Milo         6
Rusty        6
Stanley      6
Leo          6
Dave         6
Louis        5
Chester      5
Finn         5
Alfie        5
Sunny        5
Phil         5
            ..
Griswold     1
Odin         1
Marty        1
Lassie       1
Newt         1
Emmie        1
Lucky        1
Edgar        1
Shadoe       1
Zeus         1
Moofasa      1
Vinscent     1
DonDon       1
Iroh         1
Koko         1
Ronduh       1
Mack         1
Ricky        1
Dotsy        1
Rolf         1
Lorelei      1
Saydee       1
Todo         1
Milky        1
Goose        1
by           1
Trevith      1
Dylan        1
Clyde        1
Staniel      1
Name: name, Length: 931, dtype: int64

# Armazenar dados

# Analisar dados